In [1]:
from datetime import datetime

In [2]:
from xai_sdk import Client as XAICLIENT

In [3]:
from xai_sdk.chat import user, system

In [4]:
from xai_sdk.tools import x_search

In [5]:
from openai import OpenAI

In [6]:
from dotenv import load_dotenv

In [7]:
from supabase import create_client, Client

In [8]:
import anthropic

In [9]:
load_dotenv()

True

In [10]:
import os

In [11]:
import requests

In [12]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [13]:
anthropic_api_key = os.getenv("CLAUDE_API_KEY")

In [14]:
client_anthropic = anthropic.Anthropic(api_key = anthropic_api_key)

In [15]:
xai_client = XAICLIENT(os.getenv("XAI_API_KEY"))

In [16]:
import json

In [17]:
supabase_url = os.environ.get("SUPABASE_URL")

In [18]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [19]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [20]:
import base64

In [21]:
import time

In [22]:
from PIL import Image

In [23]:
import matplotlib.pyplot as plt

In [24]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
.gte('news_date', '2026-01-26')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 18,
  'news_output': 'Orbital (Jan 26, 2026) raised $60 million in a Series B led by Brighton Park Capital with investment from REV (RELX’s VC arm) and others. The funding will accelerate U.S. expansion (doubling headcount, opening hubs), scale its real-estate/legal AI products (supports ~200,000 transactions across US/UK in 2025), and deepen integrations with legal workflows. Source includes CEO interview and investor remarks.',
  'source_urls': ['https://www.artificiallawyer.com/2026/01/26/orbital-raises-60m-lexisnexis-connected-fund-invests/'],
  'news_date': '2026-01-26'},
 {'id': 19,
  'news_output': 'Algolia (Jan 26, 2026) issued a Business Wire announcement ahead of Shoptalk Luxe 2026 announcing demos of its Agent Studio platform (agentic and generative AI for retail), highlighting features to ensure accurate, real-time product data in AI shopping experiences and a collaboration to integrate retailer data into Microsoft Copilot, Bing Shopping and Edge to reduce hallucina

In [27]:
def prompt_generator_research():
    formatted_prompt = []
    
    for item in result.data:
        event_id = f"{item['id']}_{item['news_date']}"
        prompt = f"""You are given news summary: {item['news_output']} and source: {item['source_urls'][0]}, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources."""

        prompt_dict = {
            'event_id': event_id,
            'prompt': prompt
        }
        
        formatted_prompt.append(prompt_dict)

    return formatted_prompt

In [25]:
prompt_generator_research()

NameError: name 'prompt_generator_research' is not defined

In [406]:
# system_prompt = prompt_generator_research()

system prompt = ['']

In [28]:
def openai_research():
    research_output = []
    for i in system_prompt:
        
        response = client.responses.create(
            model = "gpt-4o",
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = i['prompt'],
        )

        output = response.output_text

        final_dict = {
            'event_id': i['event_id'],
            'output': output
        }

        research_output.append(final_dict)

    return research_output

In [408]:
openai_research()

[{'event_id': '18_2026-01-26',
  'output': "Here is a detailed, structured breakdown of Orbital’s January 26, 2026 Series\u202fB funding, based on multiple reputable sources:\n\n**1. Fundraise Overview**  \n- Orbital, an AI platform specializing in real estate law, announced on **January 26, 2026** that it raised **$60\u202fmillion in Series\u202fB financing**. This round was led by **Brighton Park Capital (BPC)** based in New York/Connecticut.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- The new capital brings Orbital’s **total funding to approximately $75\u202fmillion** to date.([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**2. Investor Participation**  \n- **New investors** in this round include:  \n  - **REV**, the venture capital arm of RELX (owner of LexisNexis Legal & Professional)  \n  - **The LegalTech Fund**  \n  - **Moderne Ventures**  \n  - **Grosvenor Group*

In [29]:
system_prompt = f""" 
You are a professional tech journalist for "The AI Times" newspaper.

Your task: Write a comprehensive news article covering today's AI developments.

INSTRUCTIONS:
1. Write a newspaper-quality article (800-1000 words)
2. Create an engaging headline
3. Use journalistic style: objective, clear, professional and opinionated
4. Include all the sources & links at the end and not ANY sources or links during the article (THIS IS NON NEGOTIABLE)

You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 

However, the opinion you have be your own but it should be based on the above article. 

Entire news story and its details:
"""

In [30]:
def openai_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:

        output = i['output']

        event_id = i['event_id']
        
        response = client.responses.create(
            model = "gpt-4.1",
            input = system_prompt + output,
            prompt_cache_key = "ai-times-article-builder-v1",
            prompt_cache_retention = "24h"
        )

        # cached = response.usage.cached_tokens
        # total = response.usage.prompt_tokens
        # print(f"Event ID: {event_id}: {cached}/{total} = {cached/total*100}%")

        print(response)
        # print(response.input_tokens)


        # final_output = response.output_text

        # json_format = json_formatter(final_output, "openai", event_id)

        # save_article(json_format)

        # final_news_article.append(json_format)

    return final_news_article

In [437]:
result_open_search = [{'event_id': '18_2026-01-26',
  'output': "Here is a detailed, structured breakdown of Orbital’s January 26, 2026 Series\u202fB funding, based on multiple reputable sources:\n\n**1. Fundraise Overview**  \n- Orbital, an AI platform specializing in real estate law, announced on **January 26, 2026** that it raised **$60\u202fmillion in Series\u202fB financing**. This round was led by **Brighton Park Capital (BPC)** based in New York/Connecticut.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- The new capital brings Orbital’s **total funding to approximately $75\u202fmillion** to date.([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**2. Investor Participation**  \n- **New investors** in this round include:  \n  - **REV**, the venture capital arm of RELX (owner of LexisNexis Legal & Professional)  \n  - **The LegalTech Fund**  \n  - **Moderne Ventures**  \n  - **Grosvenor Group** (an international real estate company and existing Orbital customer)([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Returning investors** include: **JLL Spark**, **Outward**, **Seedcamp**, and in some accounts **LocalGlobe** and **Parker89** (the venture capital arm of First\u202fAmerican).([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**3. Strategic Objectives for the Funding**  \n- Orbital will use the funding to:  \n  - **Accelerate expansion in the U.S.**, building on the launch of its New York office in 2025, by **doubling its headcount** and establishing **additional U.S. hubs**.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n  - **Continue growth in the U.K.** and bolster adoption across the wider real estate transaction ecosystem.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n  - **Scale its AI-powered platform**, developing a **secure, end-to-end workspace** for legal work across the full lifecycle of real estate assets.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**4. Market Position and Platform Capabilities**  \n- Orbital claims to be the **U.K. market leader in real estate legal AI**, and aims to bring that leadership into the U.S. market.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- Its platform **leverages AI tailored for real estate law**, integrating **spatial visualization, mapping, and real estate data** to automate document-heavy workflows involving records, maps, and historic deeds.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- In **2025**, Orbital supported approximately **200,000 real estate transactions across the U.S. and U.K.**([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**5. Customer Base and Reach**  \n- The company serves a broad client base including:  \n  - **AM Law\u202f100 and Magic Circle law firms**  \n  - **In-house legal teams**  \n  - **Real estate developers**, **title companies**, and **REITs**  \n  - Specific U.S. clients include **Seyfarth Shaw**, **Vinson & Elkins**, **Goodwin**, **BCLP**, and **A&O Shearman**; U.K. users include **Clifford Chance**, **Fieldfisher**, **Marks & Spencer**, and **Grosvenor Group**.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**6. Leadership and Investor Commentary**  \n- **Will Pearce**, co-founder and CEO, emphasized that real estate is the **world’s largest asset class**, yet its supporting legal work remains **manual, fragmented, and outdated**, making Orbital’s mission—modernizing it with AI—both urgent and timely.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Kevin Magan**, partner at Brighton Park Capital and new board member, noted that real estate law is **one of the most complex and underserved legal markets**, and praised Orbital’s deep domain expertise and workflow focus.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Sean Fitzpatrick**, CEO of LexisNexis North America, UK & Ireland, highlighted that REV and LexisNexis invest in customer-centric innovators like Orbital who help legal professionals deliver better outcomes.([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion?utm_source=openai))\n\n**7. Foundational Background and Evolution**  \n- Orbital was founded in **2018** (initially as “Orbital Witness”) by **Will Pearce** and **Ed Boulle**. It originally leveraged **satellite imagery and geospatial data** for property assessment before pivoting to legal AI.([artificiallawyer.com](https://www.artificiallawyer.com/2026/01/26/orbital-raises-60m-lexisnexis-connected-fund-invests/?utm_source=openai))  \n- Early funding rounds include a **2019 £1.6\u202fmillion** raise and a **2020 £4.5\u202fmillion** round, progressing to a **2023 Series A (~€8.6\u202fmillion)**, leading up to the current Series\u202fB.([legaltechnology.com](https://legaltechnology.com/2026/01/26/orbital-raises-60m-series-b-as-it-eyes-further-us-growth/?utm_source=openai))  \n- Orbital was a participant in **Mishcon de Reya’s MDR LAB** in 2017, marking one of its earliest incubations.([legaltechnology.com](https://legaltechnology.com/2026/01/26/orbital-raises-60m-series-b-as-it-eyes-further-us-growth/?utm_source=openai))\n\n**Overall Assessment**  \nThis significant $60\u202fmillion Series\u202fB raise positions Orbital at the vanguard of **legal AI tailored for real estate**, giving it the runway to scale rapidly across the U.S., enhance its product suite, and deepen its impact across complex workflows in one of the largest asset classes globally. With a team deeply rooted in domain-specific expertise and a roster of heavyweight investors, Orbital appears well-placed to define the next wave of legal automation in real estate.\n\nLet me know if you'd like deeper insights—such as competitive landscape, technical platform details, or projected growth strategy across U.S. markets."},
 {'event_id': '19_2026-01-26',
  'output': 'Below is a structured deep-dive summary of Algolia’s January\u202f26,\u202f2026 announcement ahead of Shoptalk\u202fLuxe 2026, enriched with additional context from other reliable sources. Each point is clearly cited for transparency. The summary is organized into key topics and insights.\n\n---\n\n## 1. Event Context: Shoptalk Luxe 2026 (January 27–29, 2026)\n\n- **Event Details**  \n  Algolia announced on January\u202f26,\u202f2026, that it will participate in Shoptalk\u202fLuxe 2026, held January\u202f27–29 at the Emirates Palace in Abu Dhabi. At booth F40, Algolia will demonstrate immersive, hands-on demos showcasing AI-powered shopping experiences tailored for luxury retailers. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Strategic Focus**  \n  The initiative underscores the ongoing transition in retail away from traditional website front-ends toward AI-native, multimodal, and agentic discovery—where intelligent AI agents serve as primary access points for consumer engagement. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n---\n\n## 2. Agent Studio: Algolia’s Agentic AI Platform\n\n- **Platform Capabilities**  \n  Algolia’s Agent Studio enables teams to build and deploy AI agents designed to be reliable, observable, orchestrated, and production-ready. Exhibitors at Shoptalk\u202fLuxe will showcase how this solution allows AI assistants to curate and compare products while delivering personalized, brand-aligned experiences. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Foundational Features (from September\u202f23,\u202f2025 launch)**  \n  Agent Studio positions Algolia’s retrieval engine—combining hybrid keyword and vector search, rules, and personalization—at the heart of the agent workflow. It supports "bring-your-own" LLMs, tool orchestration, observability (traces, evaluation harnesses, A/B testing), seamless UI integration, and enterprise-grade readiness (real-time indexing, flexible schema, audit trails). ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai))\n\n- **Objective**  \n  Agent Studio addresses key limitations in agentic application development: outdated or inaccurate retrieval, lack of transparency, brittle prototyping, and difficulty scaling to production. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai))\n\n---\n\n## 3. Microsoft Collaboration: Real-Time, Accurate Retail Product Data\n\n- **Partnership Overview (January\u202f12,\u202f2026)**  \n  Algolia announced a collaboration with Microsoft to integrate its real-time, enriched product data—including pricing, availability, and other attributes—into Microsoft Copilot, Bing Shopping, and Edge. This integration is designed to give retailers greater control over how their products are presented in AI-powered shopping interfaces. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n- **Benefits to Retailers and Consumers**  \n  The partnership mitigates issues arising from stale or inaccurate product feeds (hallucinations), enhancing brand visibility, trust, and discoverability across emerging AI surfaces. Early pilot collaborators include Frasers Group, JTV, Little Sleepies, Shoe Carnival, and Shoe Station. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n- **Industry Significance**  \n  With growing consumer reliance on AI tools for shopping—nearly 60% of U.S. consumers according to Algolia—this collaboration marks a pivotal shift toward agentic commerce where first impressions occur off-site in AI environments. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n---\n\n## 4. Broader Retail AI Trends and Industry Alignment\n\n- **Retail’s AI Embrace at NRF 2026**  \n  At NRF 2026 (January\u202f11–13, New York), Algolia\'s Bernadette Nixon joined a panel alongside executives from Frasers Group and Microsoft to discuss how AI agents and adaptive commerce are reshaping retail. The panel emphasized the end of traditional website storefronts in favor of AI-mediated consumer journeys. ([algolia.com](https://www.algolia.com/about/news/beyond-the-website-algolia-takes-on-the-next-era-of-ai-driven-retail-at-nrf-2026?utm_source=openai))\n\n- **Microsoft’s Expanded Retail Agentic Offerings**  \n  In parallel, Microsoft has launched a suite of agentic tools across retail operations:\n    - *Copilot Checkout*: Allows in-chat purchases within Copilot without redirects (U.S. rollout).\n    - *Brand Agents*: On Shopify, AI assistants trained on brand catalogs that speak in the retailer’s voice.\n    - *Agent Templates*: For shopping, catalog enrichment, and store operations—enhancing discovery, personalization, and back-end efficiency.\n  These developments align with Algolia’s strategy to feed product data into AI systems controlling shopping experiences across platforms. ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai))\n\n---\n\n## 5. Why It Matters: Implications for Retail and AI\n\n- **Shifting Consumer Behavior**  \n  AI is reshaping shopping, with consumers increasingly relying on agents to compare, validate, and decide—even before visiting retailer websites. Algolia’s survey found that:\n    - 33% of U.S. consumers use generative AI to validate high-value (“splurge”) purchases.\n    - 53% use AI image search for informed luxury buying decisions. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Addressing AI Hallucinations and Control Gaps**  \n  With off-site AI surfaces becoming primary discovery channels, brands risk misrepresentation unless they actively manage product data. Algolia’s technology ensures real-time, retailer-approved data—curbing hallucinations and stale feeds. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Strategic Advantage for Luxury Retailers**  \n  At Shoptalk\u202fLuxe, Algolia’s demos are positioned to showcase how luxury brands can maintain brand integrity while leveraging AI, a key differentiator in a crowded, AI-native retail landscape. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n---\n\n## Summary Table\n\n| Topic                 | Highlights |\n|----------------------|------------|\n| Event                | Algolia to demo at Shoptalk\u202fLuxe Jan\u202f27–29, 2026, in Abu Dhabi ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai)) |\n| Agent Studio         | Launched Sept\u202f23,\u202f2025; supports scalable, trustworthy agentic AI with real-time retrieval, observability, and orchestration ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai)) |\n| Microsoft Collaboration | Real-time product data integration starting Jan\u202f12, 2026; reduces hallucinations and enhances brand control across Copilot, Bing Shopping, Edge ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai)) |\n| Market Trends        | Growing consumer use of AI for shopping validation and discovery; agents evolving as first point of contact ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai)) |\n| Industry Context     | Microsoft expanding agentic retail via checkout, brand agents, templates; aligns with Algolia’s data strategy ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai)) |\n\n---\n\nThese developments reflect a broader transformation in retail: AI agents are increasingly central to discovery, and platforms like Algolia and Microsoft are enabling brands to meet that shift with accuracy, control, and operational confidence.\n\nLet me know if you’d like a deeper analysis on Agent Studio’s tech, Algolia’s competitive positioning, or how retailers can operationalize these capabilities.'}]

openai_article_builder(result_open_search)

Response(id='resp_0f7ce14a7c58cace00697998b2fc00819093d659bcb414904c', created_at=1769576627.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_0f7ce14a7c58cace00697998b3a0c481909ff2c3e332330ec2', content=[ResponseOutputText(annotations=[], text='**Orbital Charts a Bold Path Forward: $60 Million Series B Underscores the Rise of Domain-Specific Legal AI**\n\n*By [Your Name], Technology Correspondent, The AI Times — January 26, 2026*\n\nIn a dramatic signal of artificial intelligence’s transformative power across legacy industries, Orbital, the U.K.-born legal technology company specializing in real estate law, today announced a $60 million Series B funding round. Orchestrated by U.S. growth equity heavyweight Brighton Park Capital (BPC), the deal elevates Orbital’s total funding to a formidable $75 million and paves the way for breakneck expansion into the world’s largest real estat

[]

In [31]:
def anthropic_research():
    research_ouput_anthropic = []
    for i in system_prompt:
        response = client_anthropic.messages.create(
            model = "claude-haiku-4-5-20251001",
            max_tokens = 4096,
            messages = [
                {
                    "role": "user",
                    "content": f"{i['prompt']}"
                }
            ],
            tools = [
                {
                    "type": "web_search_20250305",
                    "name": "web_search",
                    "max_uses": 5
                }
            ]
        )

        final_dict = {
            'event_id': i['event_id'],
            'blocks': response.content
        }

        research_ouput_anthropic.append(final_dict)

        time.sleep(60)

    return research_ouput_anthropic

In [32]:
def extract_final_text(blocks):
    last_tool_idx = -1
    for i,b in enumerate(blocks):
        if getattr(b, "type", None) in ("server_tool_use", "web_search_tool_result"):
            last_tool_idx = i

    text_parts = []
    for b in blocks[last_tool_idx + 1:]:
        if getattr(b, "type", None) == "text":
            text_parts.append(b.text)

    return "\n".join(text_parts).strip()

In [33]:
def extract_sources(blocks):
    sources = []

    for b in blocks:
        if getattr(b, "type", None) == "web_search_tool_result":
            for r in getattr(b, "results", []):
                sources.append({
                    "title": r.get("title"),
                    "url": r.get("url"),
                    "source": r.get("source")
                })

    return sources

In [34]:
def final_claude_responses():
    results = anthropic_research()
    
    claude_responses = []
    
    for item in results:
        event_id = item['event_id']
        blocks = item['blocks']
        
        story = extract_final_text(blocks)
        sources = extract_sources(blocks)           

        
        sources_text = "\n".join(
            f"- {s.get('title', '(no title)')}\n  {s.get('url', '')}"
            for s in sources
            if s.get("url")
        ).strip()

        
        payload = f"{story}".strip()

        print(payload)

        if sources_text:
            payload += "\n\nSOURCES:\n" + sources_text

    
        if payload.strip():
            claude_responses.append({
                "event_id": event_id,
                "output": payload
            })

    return claude_responses

In [416]:
result_open_search = [{'event_id': '18_2026-01-26',
  'output': "Here is a detailed, structured breakdown of Orbital’s January 26, 2026 Series\u202fB funding, based on multiple reputable sources:\n\n**1. Fundraise Overview**  \n- Orbital, an AI platform specializing in real estate law, announced on **January 26, 2026** that it raised **$60\u202fmillion in Series\u202fB financing**. This round was led by **Brighton Park Capital (BPC)** based in New York/Connecticut.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- The new capital brings Orbital’s **total funding to approximately $75\u202fmillion** to date.([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**2. Investor Participation**  \n- **New investors** in this round include:  \n  - **REV**, the venture capital arm of RELX (owner of LexisNexis Legal & Professional)  \n  - **The LegalTech Fund**  \n  - **Moderne Ventures**  \n  - **Grosvenor Group** (an international real estate company and existing Orbital customer)([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Returning investors** include: **JLL Spark**, **Outward**, **Seedcamp**, and in some accounts **LocalGlobe** and **Parker89** (the venture capital arm of First\u202fAmerican).([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**3. Strategic Objectives for the Funding**  \n- Orbital will use the funding to:  \n  - **Accelerate expansion in the U.S.**, building on the launch of its New York office in 2025, by **doubling its headcount** and establishing **additional U.S. hubs**.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n  - **Continue growth in the U.K.** and bolster adoption across the wider real estate transaction ecosystem.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n  - **Scale its AI-powered platform**, developing a **secure, end-to-end workspace** for legal work across the full lifecycle of real estate assets.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**4. Market Position and Platform Capabilities**  \n- Orbital claims to be the **U.K. market leader in real estate legal AI**, and aims to bring that leadership into the U.S. market.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- Its platform **leverages AI tailored for real estate law**, integrating **spatial visualization, mapping, and real estate data** to automate document-heavy workflows involving records, maps, and historic deeds.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- In **2025**, Orbital supported approximately **200,000 real estate transactions across the U.S. and U.K.**([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**5. Customer Base and Reach**  \n- The company serves a broad client base including:  \n  - **AM Law\u202f100 and Magic Circle law firms**  \n  - **In-house legal teams**  \n  - **Real estate developers**, **title companies**, and **REITs**  \n  - Specific U.S. clients include **Seyfarth Shaw**, **Vinson & Elkins**, **Goodwin**, **BCLP**, and **A&O Shearman**; U.K. users include **Clifford Chance**, **Fieldfisher**, **Marks & Spencer**, and **Grosvenor Group**.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**6. Leadership and Investor Commentary**  \n- **Will Pearce**, co-founder and CEO, emphasized that real estate is the **world’s largest asset class**, yet its supporting legal work remains **manual, fragmented, and outdated**, making Orbital’s mission—modernizing it with AI—both urgent and timely.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Kevin Magan**, partner at Brighton Park Capital and new board member, noted that real estate law is **one of the most complex and underserved legal markets**, and praised Orbital’s deep domain expertise and workflow focus.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Sean Fitzpatrick**, CEO of LexisNexis North America, UK & Ireland, highlighted that REV and LexisNexis invest in customer-centric innovators like Orbital who help legal professionals deliver better outcomes.([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion?utm_source=openai))\n\n**7. Foundational Background and Evolution**  \n- Orbital was founded in **2018** (initially as “Orbital Witness”) by **Will Pearce** and **Ed Boulle**. It originally leveraged **satellite imagery and geospatial data** for property assessment before pivoting to legal AI.([artificiallawyer.com](https://www.artificiallawyer.com/2026/01/26/orbital-raises-60m-lexisnexis-connected-fund-invests/?utm_source=openai))  \n- Early funding rounds include a **2019 £1.6\u202fmillion** raise and a **2020 £4.5\u202fmillion** round, progressing to a **2023 Series A (~€8.6\u202fmillion)**, leading up to the current Series\u202fB.([legaltechnology.com](https://legaltechnology.com/2026/01/26/orbital-raises-60m-series-b-as-it-eyes-further-us-growth/?utm_source=openai))  \n- Orbital was a participant in **Mishcon de Reya’s MDR LAB** in 2017, marking one of its earliest incubations.([legaltechnology.com](https://legaltechnology.com/2026/01/26/orbital-raises-60m-series-b-as-it-eyes-further-us-growth/?utm_source=openai))\n\n**Overall Assessment**  \nThis significant $60\u202fmillion Series\u202fB raise positions Orbital at the vanguard of **legal AI tailored for real estate**, giving it the runway to scale rapidly across the U.S., enhance its product suite, and deepen its impact across complex workflows in one of the largest asset classes globally. With a team deeply rooted in domain-specific expertise and a roster of heavyweight investors, Orbital appears well-placed to define the next wave of legal automation in real estate.\n\nLet me know if you'd like deeper insights—such as competitive landscape, technical platform details, or projected growth strategy across U.S. markets."},
 {'event_id': '19_2026-01-26',
  'output': 'Below is a structured deep-dive summary of Algolia’s January\u202f26,\u202f2026 announcement ahead of Shoptalk\u202fLuxe 2026, enriched with additional context from other reliable sources. Each point is clearly cited for transparency. The summary is organized into key topics and insights.\n\n---\n\n## 1. Event Context: Shoptalk Luxe 2026 (January 27–29, 2026)\n\n- **Event Details**  \n  Algolia announced on January\u202f26,\u202f2026, that it will participate in Shoptalk\u202fLuxe 2026, held January\u202f27–29 at the Emirates Palace in Abu Dhabi. At booth F40, Algolia will demonstrate immersive, hands-on demos showcasing AI-powered shopping experiences tailored for luxury retailers. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Strategic Focus**  \n  The initiative underscores the ongoing transition in retail away from traditional website front-ends toward AI-native, multimodal, and agentic discovery—where intelligent AI agents serve as primary access points for consumer engagement. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n---\n\n## 2. Agent Studio: Algolia’s Agentic AI Platform\n\n- **Platform Capabilities**  \n  Algolia’s Agent Studio enables teams to build and deploy AI agents designed to be reliable, observable, orchestrated, and production-ready. Exhibitors at Shoptalk\u202fLuxe will showcase how this solution allows AI assistants to curate and compare products while delivering personalized, brand-aligned experiences. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Foundational Features (from September\u202f23,\u202f2025 launch)**  \n  Agent Studio positions Algolia’s retrieval engine—combining hybrid keyword and vector search, rules, and personalization—at the heart of the agent workflow. It supports "bring-your-own" LLMs, tool orchestration, observability (traces, evaluation harnesses, A/B testing), seamless UI integration, and enterprise-grade readiness (real-time indexing, flexible schema, audit trails). ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai))\n\n- **Objective**  \n  Agent Studio addresses key limitations in agentic application development: outdated or inaccurate retrieval, lack of transparency, brittle prototyping, and difficulty scaling to production. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai))\n\n---\n\n## 3. Microsoft Collaboration: Real-Time, Accurate Retail Product Data\n\n- **Partnership Overview (January\u202f12,\u202f2026)**  \n  Algolia announced a collaboration with Microsoft to integrate its real-time, enriched product data—including pricing, availability, and other attributes—into Microsoft Copilot, Bing Shopping, and Edge. This integration is designed to give retailers greater control over how their products are presented in AI-powered shopping interfaces. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n- **Benefits to Retailers and Consumers**  \n  The partnership mitigates issues arising from stale or inaccurate product feeds (hallucinations), enhancing brand visibility, trust, and discoverability across emerging AI surfaces. Early pilot collaborators include Frasers Group, JTV, Little Sleepies, Shoe Carnival, and Shoe Station. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n- **Industry Significance**  \n  With growing consumer reliance on AI tools for shopping—nearly 60% of U.S. consumers according to Algolia—this collaboration marks a pivotal shift toward agentic commerce where first impressions occur off-site in AI environments. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n---\n\n## 4. Broader Retail AI Trends and Industry Alignment\n\n- **Retail’s AI Embrace at NRF 2026**  \n  At NRF 2026 (January\u202f11–13, New York), Algolia\'s Bernadette Nixon joined a panel alongside executives from Frasers Group and Microsoft to discuss how AI agents and adaptive commerce are reshaping retail. The panel emphasized the end of traditional website storefronts in favor of AI-mediated consumer journeys. ([algolia.com](https://www.algolia.com/about/news/beyond-the-website-algolia-takes-on-the-next-era-of-ai-driven-retail-at-nrf-2026?utm_source=openai))\n\n- **Microsoft’s Expanded Retail Agentic Offerings**  \n  In parallel, Microsoft has launched a suite of agentic tools across retail operations:\n    - *Copilot Checkout*: Allows in-chat purchases within Copilot without redirects (U.S. rollout).\n    - *Brand Agents*: On Shopify, AI assistants trained on brand catalogs that speak in the retailer’s voice.\n    - *Agent Templates*: For shopping, catalog enrichment, and store operations—enhancing discovery, personalization, and back-end efficiency.\n  These developments align with Algolia’s strategy to feed product data into AI systems controlling shopping experiences across platforms. ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai))\n\n---\n\n## 5. Why It Matters: Implications for Retail and AI\n\n- **Shifting Consumer Behavior**  \n  AI is reshaping shopping, with consumers increasingly relying on agents to compare, validate, and decide—even before visiting retailer websites. Algolia’s survey found that:\n    - 33% of U.S. consumers use generative AI to validate high-value (“splurge”) purchases.\n    - 53% use AI image search for informed luxury buying decisions. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Addressing AI Hallucinations and Control Gaps**  \n  With off-site AI surfaces becoming primary discovery channels, brands risk misrepresentation unless they actively manage product data. Algolia’s technology ensures real-time, retailer-approved data—curbing hallucinations and stale feeds. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Strategic Advantage for Luxury Retailers**  \n  At Shoptalk\u202fLuxe, Algolia’s demos are positioned to showcase how luxury brands can maintain brand integrity while leveraging AI, a key differentiator in a crowded, AI-native retail landscape. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n---\n\n## Summary Table\n\n| Topic                 | Highlights |\n|----------------------|------------|\n| Event                | Algolia to demo at Shoptalk\u202fLuxe Jan\u202f27–29, 2026, in Abu Dhabi ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai)) |\n| Agent Studio         | Launched Sept\u202f23,\u202f2025; supports scalable, trustworthy agentic AI with real-time retrieval, observability, and orchestration ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai)) |\n| Microsoft Collaboration | Real-time product data integration starting Jan\u202f12, 2026; reduces hallucinations and enhances brand control across Copilot, Bing Shopping, Edge ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai)) |\n| Market Trends        | Growing consumer use of AI for shopping validation and discovery; agents evolving as first point of contact ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai)) |\n| Industry Context     | Microsoft expanding agentic retail via checkout, brand agents, templates; aligns with Algolia’s data strategy ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai)) |\n\n---\n\nThese developments reflect a broader transformation in retail: AI agents are increasingly central to discovery, and platforms like Algolia and Microsoft are enabling brands to meet that shift with accuracy, control, and operational confidence.\n\nLet me know if you’d like a deeper analysis on Agent Studio’s tech, Algolia’s competitive positioning, or how retailers can operationalize these capabilities.'}]

openai_article_builder(result_open_search)

Perfect! Now I have comprehensive information to create a well-structured research document. Let me compile this into a detailed report.

---

# ORBITAL SERIES B FUNDING: COMPREHENSIVE RESEARCH REPORT
## January 26, 2026 | $60 Million Series B Round

---

## EXECUTIVE SUMMARY


London-based real estate legal AI platform Orbital has raised $60 million in Series B funding to expand its presence in the U.S. market and build what it envisions as a comprehensive workspace for real estate legal work.
 
The funding brings Orbital's total capital raised to $75 million since its founding in 2018.


---

## I. FUNDING OVERVIEW

### Round Details
- **Amount:** 
$60 million Series B funding round

- **Lead Investor:** 
New York-based growth fund Brighton Park Capital

- **Date:** January 26, 2026
- **Total Funding to Date:** 
$75 million since its founding in 2018


### Investor Syndicate

**New Investors:**
- 
REV, the venture capital arm of RELX, which owns LexisNexis Legal and Professional

- 


[{'event_id': '18_2026-01-26',
  'output': 'Perfect! Now I have comprehensive information to create a well-structured research document. Let me compile this into a detailed report.\n\n---\n\n# ORBITAL SERIES B FUNDING: COMPREHENSIVE RESEARCH REPORT\n## January 26, 2026 | $60 Million Series B Round\n\n---\n\n## EXECUTIVE SUMMARY\n\n\nLondon-based real estate legal AI platform Orbital has raised $60 million in Series B funding to expand its presence in the U.S. market and build what it envisions as a comprehensive workspace for real estate legal work.\n \nThe funding brings Orbital\'s total capital raised to $75 million since its founding in 2018.\n\n\n---\n\n## I. FUNDING OVERVIEW\n\n### Round Details\n- **Amount:** \n$60 million Series B funding round\n\n- **Lead Investor:** \nNew York-based growth fund Brighton Park Capital\n\n- **Date:** January 26, 2026\n- **Total Funding to Date:** \n$75 million since its founding in 2018\n\n\n### Investor Syndicate\n\n**New Investors:**\n- \nREV, 

In [35]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 1800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a professional tech journalist for "The AI Times" newspaper.
                
                Your task: Write a comprehensive news article covering today's AI developments.
                Entire news story and its details: {i['output']}
                
                INSTRUCTIONS:
                1. Write a newspaper-quality article (800-1000 words)
                2. Create an engaging headline
                3. Use journalistic style: objective, clear, professional and opinionated
                4. Include all the sources & links at the end and not ANY sources or links during the article (THIS IS NON NEGOTIABLE)
                
                You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
                
                However, the opinion you have be your own but it should be based on the above article. 
            """
            }]
        )

        event_id = i['event_id']

        final_output = response.content[0].text

        json_format = json_formatter(final_output, "claude",event_id)

        save_article(json_format)

        final_news_article.append(json_format)

    return final_news_article

In [419]:
claude_article_builder(search_result_claude)

[{'model': 'claude',
  'headline': "London's Orbital Raises $60M to Revolutionize Real Estate Law with AI-Powered Automation",
  'content': '**New York growth fund leads Series B as European legaltech upstart targets US expansion with specialized platform addressing \'dramatically under-automated\' property transactions**\n\n---\n\nThe real estate legal sector, long criticized for relying on 19th-century workflows to manage 21st-century transactions, is finally getting its technological reckoning. London-based Orbital has secured $60 million in Series B funding to expand its AI-powered platform that promises to transform how property transactions are conducted across the United States and United Kingdom.\n\nThe funding round, led by New York-based Brighton Park Capital, brings Orbital\'s total capital raised to $75 million since its 2018 founding and signals growing investor appetite for specialized legal AI solutions that tackle specific practice areas rather than offering generalized

In [36]:
def json_formatter(final_output, model, event_id):

    json_format = []
    
    response_json = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [
                {"role": "system", "content": f"""Your job is to look at the entire output of 
                {final_output} and convert into a STRICT JSON format
                
                headline is pretty straightforward to determine
                content: needs to be the full content after the headline
                sources: are the links and sources from which the article is created
                model & event id would be given in the input in the form of {model} and {event_id} respectively, so just paste that in the
                JSON format

                YOU ARE NOT SUPPOSED TO UPDATE ANYTHING ON YOUR OWN, YOU NEED TO JUST TRANSFORM THE CONTENT INTO A JSON FORMAT
                """},
                {"role": "assistant", "content": f""" 
                {{
                  "model": "{model}",
                  "headline": "<headline here>",
                  "content": "<full article content here>",
                  "sources": ["<source here>"],
                  "event_id": "{event_id}"
                }}
                """}
            ]
        )

    raw_response = response_json.choices[0].message.content
    
    clean_json = raw_response.replace("```json", "").replace("```", "").strip()

    article_data_cleaned = json.loads(clean_json)
    
    return article_data_cleaned

In [37]:
def save_article(article_json):
    supabase.table('articles').insert({
        'event_id': article_json['event_id'],
        'model': article_json['model'],
        'headline': article_json['headline'],
        'content': article_json['content'],
        'sources': json.dumps(article_json['sources']),
        'word_count': len(article_json['content'].split())
    }).execute()

In [38]:
def openai_research_final(data):
    research_output = []
    for i in data_1:
        
        response = client.responses.create(
            model = "gpt-5-nano",
            reasoning = {"effort": "high"},
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = i['prompt'],
        )

        output = response.output_text

        final_dict = {
            'event_id': i['event_id'],
            'output': output
        }

        research_output.append(final_dict)

    return research_output

In [447]:
data_1 = [{'event_id': '18_2026-01-26',
  'prompt': 'You are given news summary: Orbital (Jan 26, 2026) raised $60 million in a Series B led by Brighton Park Capital with investment from REV (RELX’s VC arm) and others. The funding will accelerate U.S. expansion (doubling headcount, opening hubs), scale its real-estate/legal AI products (supports ~200,000 transactions across US/UK in 2025), and deepen integrations with legal workflows. Source includes CEO interview and investor remarks. and source: https://www.artificiallawyer.com/2026/01/26/orbital-raises-60m-lexisnexis-connected-fund-invests/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.'},
 {'event_id': '19_2026-01-26',
  'prompt': 'You are given news summary: Algolia (Jan 26, 2026) issued a Business Wire announcement ahead of Shoptalk Luxe 2026 announcing demos of its Agent Studio platform (agentic and generative AI for retail), highlighting features to ensure accurate, real-time product data in AI shopping experiences and a collaboration to integrate retailer data into Microsoft Copilot, Bing Shopping and Edge to reduce hallucinations and keep brand control. and source: https://sg.finance.yahoo.com/news/algolia-showcases-future-ai-driven-050000722.html, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.'}]


openai_research_final(data_1)

[{'event_id': '18_2026-01-26',
  'output': 'Here is a concise, sourced briefing on Orbital’s $60M Series B and what it means, with key dates anchored to exact calendar days.\n\nExecutive snapshot (as of Jan 30, 2026)\n- Orbital, the AI platform for real estate law, announced a $60 million Series B on January 26, 2026. The round was led by Brighton Park Capital, with participation from REV (RELX’s venture arm that owns LexisNexis), The LegalTech Fund, Moderne Ventures, Grosvenor Group, JLL Spark, Outward, and Seedcamp. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- The company described the investment as accelerating growth in the US and UK, expanding adoption across the real estate transaction ecosystem, and building a unified real-estate legal workspace. Orbital has offices in New York and London. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- Orbital ended 2025 with roughly 200,000 real estate transactions supported across the U

In [450]:
a = [{'event_id': '18_2026-01-26',
  'output': 'Here is a concise, sourced briefing on Orbital’s $60M Series B and what it means, with key dates anchored to exact calendar days.\n\nExecutive snapshot (as of Jan 30, 2026)\n- Orbital, the AI platform for real estate law, announced a $60 million Series B on January 26, 2026. The round was led by Brighton Park Capital, with participation from REV (RELX’s venture arm that owns LexisNexis), The LegalTech Fund, Moderne Ventures, Grosvenor Group, JLL Spark, Outward, and Seedcamp. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- The company described the investment as accelerating growth in the US and UK, expanding adoption across the real estate transaction ecosystem, and building a unified real-estate legal workspace. Orbital has offices in New York and London. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- Orbital ended 2025 with roughly 200,000 real estate transactions supported across the US and UK, with customers including Am Law 100/Magic Circle firms and various real-estate participants (developers, title companies, REITs). The firm intends to double headcount and open additional US hubs as part of the expansion. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- The round’s investors include both new participants (REV, The LegalTech Fund, Grosvenor Group) and existing supporters (JLL Spark, Outward, Seedcamp; Parker89 and First American’s venture arm are noted in other coverage). The total funding to date is cited as $75M by Orbital’s disclosures. ([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion))\n\nFunding details and investors\n- Amount and lead: $60M Series B led by Brighton Park Capital. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- Key investors (new and existing): REV (RELX’s LexisNexis venture arm), The LegalTech Fund, Moderne Ventures, Grosvenor Group; JLL Spark; Outward; Seedcamp. Parker89 (First American) is referenced in some coverage as an earlier investor; Orbital’s own materials emphasize REV’s participation. ([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion))\n- corroborating coverage notes similar investor rosters and the round’s purpose. See FinSMEs, The SaaS News, and LawSites summaries. ([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\nUse of proceeds and growth plan\n- Primary intent: accelerate growth in the US and UK, expand adoption across the real estate transaction ecosystem, and move toward a single secure workspace for real estate legal work across the asset lifecycle. Orbital also plans to double headcount and establish additional US hubs, following its 2025 New York presence. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- The firm frames this as expanding into a $140B opportunity in real estate law, with a focus on automating document-heavy, multi-record workflows and related legal tasks. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n\nProduct and market positioning\n- Orbital markets itself as a purpose-built AI platform for real estate law, combining specialized legal AI with spatial visualization, mapping, and real estate data to streamline title, survey, and related workflows. The aim is to replace manual, document-heavy reviews with faster, more transparent digital processes. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- Orbital’s Copilot and related products (e.g., Orbital Witness, Orbital Residential) are highlighted as enabling faster, more consistent conveyancing and property-workflows; customer use includes Am Law 100/Magic Circle firms and large real estate-related entities. ([orbital.tech](https://www.orbital.tech/blog/seyfarth-real-estate-practice-specific-ai?utm_source=openai))\n- Industry coverage corroborates the platform’s positioning and growth trajectory, including independent outlets such as FinSMEs, The SaaS News, SiliconANGLE, and LawSites. ([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\nEvidence of market momentum and ecosystem signals\n- The 2025 data point: Orbital ended 2025 supporting about 200,000 real estate transactions across the US and UK, underscoring scale and adoption to date. This mirrors information from Orbital’s own release and LexisNexis’ press coverage. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- Orbital’s UK leadership position in real estate legal AI is frequently cited, with expansion plans explicitly targeting the US market and broader real estate ecosystem. ([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion))\n- The investor syndicate includes LexisNexis’ REV, which ties Orbital to LexisNexis’ data and workflow ecosystems, potentially enabling deeper integrations with Lexis content and tools. ([lexisnexis.com](https://www.lexisnexis.com/community/amp-pressroom/rev-lexisnexis-invest-in-orbital-the-leading-ai-platform-for-real-estate-law))\n\nKey dates to anchor the timeline\n- January 26, 2026: Orbital announces a $60M Series B funding round led by Brighton Park Capital, with REV (RELX/LexisNexis) and other investors participating. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- 2025: Orbital opened a New York office and began US expansion; by year-end, the company reported supporting ~200,000 real estate transactions in the US and UK. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n\nWhat this implies for stakeholders\n- For real estate and law firms: Orbital’s momentum and investor backing reinforce a shift toward practice-specific AI for complex real estate workflows, potentially reducing deal times and increasing consistency across transactions. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- For LexisNexis and REV: The investment signals a strengthening of LexisNexis’ ecosystem strategy by backing a vendor whose platform could integrate with Lexis content and workflows. ([lexisnexis.com](https://www.lexisnexis.com/community/amp-pressroom/rev-lexisnexis-invest-in-orbital-the-leading-ai-platform-for-real-estate-law))\n- For the broader legal-tech market: The round adds to a growing cohort of real-estate–focused AI players attracting cross-industry investor syndicates, including real estate operators and specialized tech funds. ([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\nNotes and caveats\n- The figures and scenarios reflect Orbital’s communications and third-party coverage as of January 2026. If you’d like, I can pull any post-Jan 30, 2026 updates (new hires, new hubs, customer wins) and add them to this brief. The core facts here (fund amount, lead investor, major participants, 200k transactions in 2025, US/UK expansion) are consistently reported across Orbital’s own posts and multiple press outlets. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n\nSources used\n- Orbital official blog, “Announcing Our $60M Series B Fundraise” (January 26, 2026) and related funding announcement. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))\n- Brighton Park Capital press/announcement page detailing the raise and expansion plan. ([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion))\n- LexisNexis pressroom (REV & LexisNexis Invest in Orbital) confirming investors and the $140B market context. ([lexisnexis.com](https://www.lexisnexis.com/community/amp-pressroom/rev-lexisnexis-invest-in-orbital-the-leading-ai-platform-for-real-estate-law))\n- FinSMEs recap of the Series B round (investors and use of funds). ([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n- The SaaS News recap (investors and round details). ([thesaasnews.com](https://www.thesaasnews.com/news/orbital-raises-60-million-in-series-b?utm_source=openai))\n- LawSites coverage of Orbital’s Series B and US expansion implications. ([lawnext.com](https://www.lawnext.com/2026/01/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-and-fuel-u-s-expansion.html?utm_source=openai))\n- Artificial Lawyer summary and interview context (January 26, 2026) for additional color on the fundraising and LexisNexis connection. ([artificiallawyer.com](https://www.artificiallawyer.com/2026/01/26/orbital-raises-60m-lexisnexis-connected-fund-invests/))\n- Additional product context from Orbital’s own materials and related customer-use notes (e.g., Seyfarth Shaw case study). ([orbital.tech](https://www.orbital.tech/blog/seyfarth-real-estate-practice-specific-ai?utm_source=openai))\n\nIf you’d like, I can convert this into a one-page briefing for an executive briefing deck, with a slide-ready outline and a short Q&A section for due diligence.'},
 {'event_id': '19_2026-01-26',
  'output': 'Below is a structured synthesis of Algolia’s January 2026 announcements and related context, drawn from Algolia’s own releases, event pages, and reputable media coverage. I’ve organized the key points by topic/date, added brief implications, and included sources after each section.\n\n1) What Algolia announced around Shoptalk Luxe 2026 (Jan 26, 2026)\n- Announcement context and venue\n  - Algolia disclosed it would attend Shoptalk Luxe 2026 (Jan 27–29, 2026) in Abu Dhabi and would host hands-on demos at Booth F40 showing how luxury retailers can use agentic AI, generative AI, and AI search in real shopping experiences. The booth is positioned to showcase real-time, AI-powered product discovery informed by live retailer behavior and trends. Source: Algolia press release page for Shoptalk Luxe 2026. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- Demos of Agent Studio\n  - At the Shoptalk Luxe booth, Algolia will demo Agent Studio — a platform to create, test, and deploy AI agents that are reliable, observable, orchestrated, and production-ready for retail contexts. The demos emphasize agents curating and comparing products to deliver on-brand experiences without sacrificing accuracy or control. Source: same Algolia Shoptalk Luxe release. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- Real-time data and brand control in AI shopping\n  - The release highlights Algolia’s emphasis on real-time, retailer-approved product data (including inventory and pricing) in AI shopping experiences, aimed at reducing stale data and hallucinations and ensuring correct product representation across AI journeys. It also notes a collaboration with Microsoft to integrate real-time product data into Copilot, Bing Shopping, and Edge. Source: Algolia Shoptalk Luxe release. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- Rich context for luxury brands (scale and scope)\n  - The piece notes that Algolia serves more than 18,000 businesses and that about 70% of luxury brands use Algolia, underscoring the platform’s relevance for high-end retail. It also references prior NRF 2026 conversations around AI-native shopping and retail shifts. Source: Algolia Shoptalk Luxe release (context and stats). ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n\n2) The Microsoft collaboration to drive real-time product data into AI shopping (Jan 12, 2026)\n- What the collaboration does\n  - Algolia announced a collaboration with Microsoft to inject Algolia’s real-time enriched product attributes (data, inventory, pricing) into Microsoft Copilot, Bing Shopping, and Edge. The goal is to help retailers ensure accurate representations across AI discovery surfaces and off-site shopping experiences. Source: Algolia press release on the Microsoft collaboration. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences))\n- Strategic rationale and quotes\n  - The release frames the collaboration as giving retailers greater influence and control over how products appear in AI-powered surfaces, addressing data quality issues that can undermine conversion when first impressions occur off retailer sites. Microsoft Shopping executives are quoted about empowering retailers to shape AI experiences rather than adapting to opaque AI systems. Source: Algolia press release (Microsoft collaboration). ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences))\n- Early pilots and partners\n  - Early Algolia customers piloting the approach include Frasers Group, JTV, Little Sleepies, Shoe Carnival, Shoe Station, among others, illustrating how real-time data alignment with AI queries can boost discoverability. Source: Microsoft collaboration release. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences))\n\n3) About Agent Studio (the product that Algolia showcased in 2025 and promoted in 2026)\n- What Agent Studio is and why it matters\n  - Agent Studio unifies retrieval, tool orchestration, and observability to enable organizations to build AI agents that run safely in production. It emphasizes grounding retrieval with enterprise data, policy governance, and production-grade evaluation, plus a “bring your own model” (BYO-LLM) approach and a Model Context Protocol (MCP) alignment. Source: Algolia press release on Agent Studio (Sept 23, 2025). ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents))\n- Key capabilities highlighted\n  - Retrieval trusted by design (hybrid keyword + vector search with rules/personalization), model-agnostic design, MCP alignment for cross-stack orchestration, tool orchestration, and ready-made UI components to embed agents in apps. These foundations are intended to address data freshness, governance, and scalability in production-grade AI agents. Source: Agent Studio release. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents))\n\n4) NRF 2026 context and broader industry framing around AI-native shopping\n- Algolia at NRF 2026\n  - Algolia positioned NRF 2026 as a venue to discuss “Beyond the website: Reinventing retail for AI-native shoppers,” with a panel including Bernadette Nixon (Algolia), David Clark (Frasers Group), and Jennifer Myers (Microsoft Shopping). The NRF session took place January 12, 2026, at the ReturnPro Stage, Javits Center, New York. Source: NRF session page and Algolia NRF coverage. ([algolia.com](https://www.algolia.com/events/event-NRF2026))\n- Industry participation and context\n  - Algolia’s NRF presence included panels and live demos (e.g., “Behind the Scenes of AI Shopping: How Algolia + Microsoft Copilot Are Redefining the Next Era of AI-Driven Product Discovery”), with cross-pollination among retailers, brands, and platform partners. This contextualizes the Shoptalk Luxe push as part of a broader industry shift toward agentic and AI-driven discovery. Source: NRF-related Algolia pages and NRF event agenda. ([algolia.com](https://www.algolia.com/events/event-NRF2026))\n- Microsoft’s NRF-related framing\n  - Microsoft highlighted agentic AI capabilities in retail around NRF 2026, including templates in Copilot Studio for catalog enrichment and store-operations automation, aligning with the broader trend Algolia is signaling with real-time product data and brand control. Source: Microsoft NRF-focused materials. ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai))\n\n5) Why this matters for retailers and what to watch next\n- What these moves enable\n  - Real-time data across AI surfaces reduces hallucinations and misrepresentation, preserving brand integrity as shoppers encounter AI-enabled discovery off retailer sites. The Microsoft collaboration explicitly targets Copilot, Bing Shopping, and Edge as surfaces where Algolia’s data can improve accuracy and discovery continuity. Sources: Shoptalk Luxe release and Microsoft collaboration release. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- Brand control and governance\n  - Algolia emphasizes that retailers can control how their products appear in AI experiences through real-time data feeds and governance mechanisms embedded in Agent Studio (e.g., memory, rules, MCP). This is described as a way to “stop reacting to AI and start driving it.” Source: Agent Studio release and Shoptalk Luxe release. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents))\n- Market context and consumer behavior\n  - Algolia notes that a large share of U.S. consumers already use AI tools for shopping, which amplifies the importance of accurate AI-driven discovery across surfaces like Copilot, Bing, Edge, and other off-site channels. Source: Microsoft collaboration release (and reiterated in Shoptalk Luxe release). ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences))\n\n6) Quick takeaways and what to expect next\n- Near-term signals\n  - The Shoptalk Luxe 2026 demos (Jan 27–29, 2026) at Booth F40 are a concrete milestone for Agent Studio’s on-product demonstrations in a live retail context. Expect post-event coverage and customer wins to be announced in early 2026. Source: Shoptalk Luxe release. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- Ongoing integration with Microsoft and others\n  - The Microsoft collaboration establishes a broader trend of real-time catalog data feeding AI copilots and browsing surfaces, setting the stage for Copilot Checkout and related AI-assisted commerce workflows. Microsoft’ NRF materials and Algolia’s Microsoft collaboration release reinforce this trajectory. Source: Algolia–Microsoft collaboration release; Microsoft NRF-related materials. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences))\n- Long-term governance and memory\n  - Agent Studio’s governance, memory capabilities, and MCP alignment suggest Algolia is aiming for enterprise-grade AI agents that persist context across sessions, critical for retail use cases like merchandising copilots and shopping assistants. Source: Agent Studio release. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents))\n\n7) Sources (selected)\n- Algolia Showcases the Future of AI-Driven Search and Discovery at Shoptalk Luxe 2026 (Algolia Newsroom) – details on Shoptalk Luxe demos, Agent Studio, and the Microsoft collaboration. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- Algolia Collaborates with Microsoft to Drive Real-Time Product Data to Shopping Experiences (Algolia Newsroom) – Microsoft Copilot, Bing Shopping, and Edge integration; early pilots. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences))\n- Algolia Launches Agent Studio to Power Scalable, Context-Aware AI Agents (Algolia Newsroom) – Agent Studio architecture, MCP, BYO-LLM, governance, production readiness. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents))\n- Beyond the website: Algolia takes on the Next Era of AI-Driven Retail at NRF 2026 (Algolia Newsroom) – NRF 2026 context and leadership. ([algolia.com](https://www.algolia.com/about/news/beyond-the-website-algolia-takes-on-the-next-era-of-ai-driven-retail-at-nrf-2026?utm_source=openai))\n- NRF 2026 – ReturnPro Stage session details and panel lineup (NRF site) – official session content and speakers. ([nrfbigshow.nrf.com](https://nrfbigshow.nrf.com/session/2542?utm_source=openai))\n- NRF 2026 – Algolia NRF schedule and in-booth demos (Algolia event page) – event agenda and demos. ([algolia.com](https://www.algolia.com/events/event-NRF2026))\n- Shoptalk Luxe 2026 (Algolia Newsroom) – event details and booth information (alternate view to the direct Shoptalk Luxe release). ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026))\n- DestinationCRM – coverage of Algolia–Microsoft collaboration (contextual industry reaction). ([destinationcrm.com](https://www.destinationcrm.com/Articles/CRM-News/CRM-Across-the-Wire/Algolia-Partners-with-Microsoft-to-Drive-Real-Time-Product-Data--173045.aspx?utm_source=openai))\n- Microsoft News – Microsoft’s NRF 2026 retail agentic AI capabilities and Copilot context (additional context for the broader AI retail push). ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai))\n\nIf you’d like, I can turn this into a one-page briefing for a meeting or a slide-ready summary with a compact timeline and direct quotes from each source.'}]


for i in a:
    print(i['output'])

Here is a concise, sourced briefing on Orbital’s $60M Series B and what it means, with key dates anchored to exact calendar days.

Executive snapshot (as of Jan 30, 2026)
- Orbital, the AI platform for real estate law, announced a $60 million Series B on January 26, 2026. The round was led by Brighton Park Capital, with participation from REV (RELX’s venture arm that owns LexisNexis), The LegalTech Fund, Moderne Ventures, Grosvenor Group, JLL Spark, Outward, and Seedcamp. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))
- The company described the investment as accelerating growth in the US and UK, expanding adoption across the real estate transaction ecosystem, and building a unified real-estate legal workspace. Orbital has offices in New York and London. ([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b))
- Orbital ended 2025 with roughly 200,000 real estate transactions supported across the US and UK, with customers including Am Law 100/Mag

In [451]:
b = [{'event_id': '18_2026-01-26',
  'output': "Here is a detailed, structured breakdown of Orbital’s January 26, 2026 Series\u202fB funding, based on multiple reputable sources:\n\n**1. Fundraise Overview**  \n- Orbital, an AI platform specializing in real estate law, announced on **January 26, 2026** that it raised **$60\u202fmillion in Series\u202fB financing**. This round was led by **Brighton Park Capital (BPC)** based in New York/Connecticut.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- The new capital brings Orbital’s **total funding to approximately $75\u202fmillion** to date.([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**2. Investor Participation**  \n- **New investors** in this round include:  \n  - **REV**, the venture capital arm of RELX (owner of LexisNexis Legal & Professional)  \n  - **The LegalTech Fund**  \n  - **Moderne Ventures**  \n  - **Grosvenor Group** (an international real estate company and existing Orbital customer)([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Returning investors** include: **JLL Spark**, **Outward**, **Seedcamp**, and in some accounts **LocalGlobe** and **Parker89** (the venture capital arm of First\u202fAmerican).([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))\n\n**3. Strategic Objectives for the Funding**  \n- Orbital will use the funding to:  \n  - **Accelerate expansion in the U.S.**, building on the launch of its New York office in 2025, by **doubling its headcount** and establishing **additional U.S. hubs**.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n  - **Continue growth in the U.K.** and bolster adoption across the wider real estate transaction ecosystem.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n  - **Scale its AI-powered platform**, developing a **secure, end-to-end workspace** for legal work across the full lifecycle of real estate assets.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**4. Market Position and Platform Capabilities**  \n- Orbital claims to be the **U.K. market leader in real estate legal AI**, and aims to bring that leadership into the U.S. market.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- Its platform **leverages AI tailored for real estate law**, integrating **spatial visualization, mapping, and real estate data** to automate document-heavy workflows involving records, maps, and historic deeds.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- In **2025**, Orbital supported approximately **200,000 real estate transactions across the U.S. and U.K.**([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**5. Customer Base and Reach**  \n- The company serves a broad client base including:  \n  - **AM Law\u202f100 and Magic Circle law firms**  \n  - **In-house legal teams**  \n  - **Real estate developers**, **title companies**, and **REITs**  \n  - Specific U.S. clients include **Seyfarth Shaw**, **Vinson & Elkins**, **Goodwin**, **BCLP**, and **A&O Shearman**; U.K. users include **Clifford Chance**, **Fieldfisher**, **Marks & Spencer**, and **Grosvenor Group**.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))\n\n**6. Leadership and Investor Commentary**  \n- **Will Pearce**, co-founder and CEO, emphasized that real estate is the **world’s largest asset class**, yet its supporting legal work remains **manual, fragmented, and outdated**, making Orbital’s mission—modernizing it with AI—both urgent and timely.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Kevin Magan**, partner at Brighton Park Capital and new board member, noted that real estate law is **one of the most complex and underserved legal markets**, and praised Orbital’s deep domain expertise and workflow focus.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  \n- **Sean Fitzpatrick**, CEO of LexisNexis North America, UK & Ireland, highlighted that REV and LexisNexis invest in customer-centric innovators like Orbital who help legal professionals deliver better outcomes.([bpc.com](https://www.bpc.com/insights/orbital-raises-60m-series-b-to-automate-real-estate-law-with-ai-amid-rapid-us-expansion?utm_source=openai))\n\n**7. Foundational Background and Evolution**  \n- Orbital was founded in **2018** (initially as “Orbital Witness”) by **Will Pearce** and **Ed Boulle**. It originally leveraged **satellite imagery and geospatial data** for property assessment before pivoting to legal AI.([artificiallawyer.com](https://www.artificiallawyer.com/2026/01/26/orbital-raises-60m-lexisnexis-connected-fund-invests/?utm_source=openai))  \n- Early funding rounds include a **2019 £1.6\u202fmillion** raise and a **2020 £4.5\u202fmillion** round, progressing to a **2023 Series A (~€8.6\u202fmillion)**, leading up to the current Series\u202fB.([legaltechnology.com](https://legaltechnology.com/2026/01/26/orbital-raises-60m-series-b-as-it-eyes-further-us-growth/?utm_source=openai))  \n- Orbital was a participant in **Mishcon de Reya’s MDR LAB** in 2017, marking one of its earliest incubations.([legaltechnology.com](https://legaltechnology.com/2026/01/26/orbital-raises-60m-series-b-as-it-eyes-further-us-growth/?utm_source=openai))\n\n**Overall Assessment**  \nThis significant $60\u202fmillion Series\u202fB raise positions Orbital at the vanguard of **legal AI tailored for real estate**, giving it the runway to scale rapidly across the U.S., enhance its product suite, and deepen its impact across complex workflows in one of the largest asset classes globally. With a team deeply rooted in domain-specific expertise and a roster of heavyweight investors, Orbital appears well-placed to define the next wave of legal automation in real estate.\n\nLet me know if you'd like deeper insights—such as competitive landscape, technical platform details, or projected growth strategy across U.S. markets."},
 {'event_id': '19_2026-01-26',
  'output': 'Below is a structured deep-dive summary of Algolia’s January\u202f26,\u202f2026 announcement ahead of Shoptalk\u202fLuxe 2026, enriched with additional context from other reliable sources. Each point is clearly cited for transparency. The summary is organized into key topics and insights.\n\n---\n\n## 1. Event Context: Shoptalk Luxe 2026 (January 27–29, 2026)\n\n- **Event Details**  \n  Algolia announced on January\u202f26,\u202f2026, that it will participate in Shoptalk\u202fLuxe 2026, held January\u202f27–29 at the Emirates Palace in Abu Dhabi. At booth F40, Algolia will demonstrate immersive, hands-on demos showcasing AI-powered shopping experiences tailored for luxury retailers. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Strategic Focus**  \n  The initiative underscores the ongoing transition in retail away from traditional website front-ends toward AI-native, multimodal, and agentic discovery—where intelligent AI agents serve as primary access points for consumer engagement. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n---\n\n## 2. Agent Studio: Algolia’s Agentic AI Platform\n\n- **Platform Capabilities**  \n  Algolia’s Agent Studio enables teams to build and deploy AI agents designed to be reliable, observable, orchestrated, and production-ready. Exhibitors at Shoptalk\u202fLuxe will showcase how this solution allows AI assistants to curate and compare products while delivering personalized, brand-aligned experiences. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Foundational Features (from September\u202f23,\u202f2025 launch)**  \n  Agent Studio positions Algolia’s retrieval engine—combining hybrid keyword and vector search, rules, and personalization—at the heart of the agent workflow. It supports "bring-your-own" LLMs, tool orchestration, observability (traces, evaluation harnesses, A/B testing), seamless UI integration, and enterprise-grade readiness (real-time indexing, flexible schema, audit trails). ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai))\n\n- **Objective**  \n  Agent Studio addresses key limitations in agentic application development: outdated or inaccurate retrieval, lack of transparency, brittle prototyping, and difficulty scaling to production. ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai))\n\n---\n\n## 3. Microsoft Collaboration: Real-Time, Accurate Retail Product Data\n\n- **Partnership Overview (January\u202f12,\u202f2026)**  \n  Algolia announced a collaboration with Microsoft to integrate its real-time, enriched product data—including pricing, availability, and other attributes—into Microsoft Copilot, Bing Shopping, and Edge. This integration is designed to give retailers greater control over how their products are presented in AI-powered shopping interfaces. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n- **Benefits to Retailers and Consumers**  \n  The partnership mitigates issues arising from stale or inaccurate product feeds (hallucinations), enhancing brand visibility, trust, and discoverability across emerging AI surfaces. Early pilot collaborators include Frasers Group, JTV, Little Sleepies, Shoe Carnival, and Shoe Station. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n- **Industry Significance**  \n  With growing consumer reliance on AI tools for shopping—nearly 60% of U.S. consumers according to Algolia—this collaboration marks a pivotal shift toward agentic commerce where first impressions occur off-site in AI environments. ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai))\n\n---\n\n## 4. Broader Retail AI Trends and Industry Alignment\n\n- **Retail’s AI Embrace at NRF 2026**  \n  At NRF 2026 (January\u202f11–13, New York), Algolia\'s Bernadette Nixon joined a panel alongside executives from Frasers Group and Microsoft to discuss how AI agents and adaptive commerce are reshaping retail. The panel emphasized the end of traditional website storefronts in favor of AI-mediated consumer journeys. ([algolia.com](https://www.algolia.com/about/news/beyond-the-website-algolia-takes-on-the-next-era-of-ai-driven-retail-at-nrf-2026?utm_source=openai))\n\n- **Microsoft’s Expanded Retail Agentic Offerings**  \n  In parallel, Microsoft has launched a suite of agentic tools across retail operations:\n    - *Copilot Checkout*: Allows in-chat purchases within Copilot without redirects (U.S. rollout).\n    - *Brand Agents*: On Shopify, AI assistants trained on brand catalogs that speak in the retailer’s voice.\n    - *Agent Templates*: For shopping, catalog enrichment, and store operations—enhancing discovery, personalization, and back-end efficiency.\n  These developments align with Algolia’s strategy to feed product data into AI systems controlling shopping experiences across platforms. ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai))\n\n---\n\n## 5. Why It Matters: Implications for Retail and AI\n\n- **Shifting Consumer Behavior**  \n  AI is reshaping shopping, with consumers increasingly relying on agents to compare, validate, and decide—even before visiting retailer websites. Algolia’s survey found that:\n    - 33% of U.S. consumers use generative AI to validate high-value (“splurge”) purchases.\n    - 53% use AI image search for informed luxury buying decisions. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Addressing AI Hallucinations and Control Gaps**  \n  With off-site AI surfaces becoming primary discovery channels, brands risk misrepresentation unless they actively manage product data. Algolia’s technology ensures real-time, retailer-approved data—curbing hallucinations and stale feeds. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n- **Strategic Advantage for Luxury Retailers**  \n  At Shoptalk\u202fLuxe, Algolia’s demos are positioned to showcase how luxury brands can maintain brand integrity while leveraging AI, a key differentiator in a crowded, AI-native retail landscape. ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai))\n\n---\n\n## Summary Table\n\n| Topic                 | Highlights |\n|----------------------|------------|\n| Event                | Algolia to demo at Shoptalk\u202fLuxe Jan\u202f27–29, 2026, in Abu Dhabi ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai)) |\n| Agent Studio         | Launched Sept\u202f23,\u202f2025; supports scalable, trustworthy agentic AI with real-time retrieval, observability, and orchestration ([algolia.com](https://www.algolia.com/about/news/algolia-launches-agent-studio-to-power-scalable-context-aware-ai-agents?utm_source=openai)) |\n| Microsoft Collaboration | Real-time product data integration starting Jan\u202f12, 2026; reduces hallucinations and enhances brand control across Copilot, Bing Shopping, Edge ([algolia.com](https://www.algolia.com/about/news/algolia-collaborates-with-microsoft-to-drive-real-time-product-data-to-shopping-experiences?utm_source=openai)) |\n| Market Trends        | Growing consumer use of AI for shopping validation and discovery; agents evolving as first point of contact ([algolia.com](https://www.algolia.com/about/news/algolia-showcases-the-future-of-ai-driven-search-and-discovery-at-shoptalk-luxe-2026?utm_source=openai)) |\n| Industry Context     | Microsoft expanding agentic retail via checkout, brand agents, templates; aligns with Algolia’s data strategy ([news.microsoft.com](https://news.microsoft.com/source/2026/01/08/microsoft-propels-retail-forward-with-agentic-ai-capabilities-that-power-intelligent-automation-for-every-retail-function/?utm_source=openai)) |\n\n---\n\nThese developments reflect a broader transformation in retail: AI agents are increasingly central to discovery, and platforms like Algolia and Microsoft are enabling brands to meet that shift with accuracy, control, and operational confidence.\n\nLet me know if you’d like a deeper analysis on Agent Studio’s tech, Algolia’s competitive positioning, or how retailers can operationalize these capabilities.'}]


for i in b:
    print(i['output'])

Here is a detailed, structured breakdown of Orbital’s January 26, 2026 Series B funding, based on multiple reputable sources:

**1. Fundraise Overview**  
- Orbital, an AI platform specializing in real estate law, announced on **January 26, 2026** that it raised **$60 million in Series B financing**. This round was led by **Brighton Park Capital (BPC)** based in New York/Connecticut.([orbital.tech](https://www.orbital.tech/blog/orbital-raises-series-b?utm_source=openai))  
- The new capital brings Orbital’s **total funding to approximately $75 million** to date.([finsmes.com](https://www.finsmes.com/2026/01/orbital-raises-60m-in-series-b-funding.html?utm_source=openai))

**2. Investor Participation**  
- **New investors** in this round include:  
  - **REV**, the venture capital arm of RELX (owner of LexisNexis Legal & Professional)  
  - **The LegalTech Fund**  
  - **Moderne Ventures**  
  - **Grosvenor Group** (an international real estate company and existing Orbital customer)([orb

In [49]:
chat = xai_client.chat.create(
    model = "grok-4-1-fast",
    tools = [x_search(
        from_date = datetime(2026,1,30)
    )]
)

chat.append(user("Give me interesting insights about AI, these can be about any new & interesting startups/products or any interesting news."))

response = chat.sample()

response.content

'Here are some fresh and intriguing AI insights from the last few days (as of late January 2026), pulled from buzzing discussions on X. I\'ve focused on new startups, product launches, acquisitions, and breakthroughs that stand out for their innovation or implications:\n\n- **Apple\'s "Silent Speech" Acquisition**: Apple just snapped up Israeli startup Qai, which specializes in AI that detects "silent speech" by analyzing tiny facial skin micromovements. It can transcribe mouthed words without sound, identify speakers, gauge emotions, heart rate, and even respiration. Patented last year, this could revolutionize hands-free interfaces (think Siri upgrades or accessibility tools), but it\'s sparking privacy freakouts—some call it "creepy surveillance tech." Massive buzz with 4K+ likes on the announcement post.\n\n- **Milq AI Launch**: Fresh out today (Jan 30), cofounders from Figma announced Milq.ai—a native Mac app that lets you build stunning iOS apps using SwiftUI, Apple\'s Liquid Gla

In [52]:
import requests
from datetime import datetime, timedelta
import time

def get_stories_by_timeframe(story_type='top', days_back=1):
    
    # API endpoints
    endpoints = {
        'top': 'https://hacker-news.firebaseio.com/v0/topstories.json',
        'new': 'https://hacker-news.firebaseio.com/v0/newstories.json',
        'best': 'https://hacker-news.firebaseio.com/v0/beststories.json',
        'ask': 'https://hacker-news.firebaseio.com/v0/askstories.json',
        'show': 'https://hacker-news.firebaseio.com/v0/showstories.json',
        'job': 'https://hacker-news.firebaseio.com/v0/jobstories.json'
    }
    
    # Get story IDs
    response = requests.get(endpoints[story_type])
    story_ids = response.json()
    
    # Calculate time range
    now = datetime.now()
    cutoff_time = now - timedelta(days=days_back)
    cutoff_timestamp = int(cutoff_time.timestamp())
    
    filtered_stories = []
    
    print(f"Fetching stories from the last {days_back} day(s)...")
    print(f"Cutoff timestamp: {cutoff_timestamp} ({cutoff_time})")
    
    # Fetch and filter stories
    for i, story_id in enumerate(story_ids):
        try:
            # Get individual story details
            story_response = requests.get(
                f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            )
            story = story_response.json()
            
            if story and 'time' in story:
                story_time = story['time']
                
                # Check if story is within timeframe
                if story_time >= cutoff_timestamp:
                    filtered_stories.append(story)
                    print(f"✓ Found: {story.get('title', 'No title')[:60]}...")
                else:
                    # Stories are roughly in chronological order, so we can break early
                    print(f"\nReached stories older than {days_back} day(s). Stopping...")
                    break
            
            # Rate limiting - be nice to the API
            time.sleep(0.1)
            
            # Progress indicator
            if (i + 1) % 50 == 0:
                print(f"Processed {i + 1} stories, found {len(filtered_stories)} matches")
                
        except Exception as e:
            print(f"Error fetching story {story_id}: {e}")
            continue
    
    return filtered_stories


def get_stories_for_specific_date(target_date, story_type='top'):
    """
    Get stories from a specific date
    
    Args:
        target_date: datetime object for the target date
        story_type: 'top', 'new', 'best', 'ask', 'show', or 'job'
    
    Returns:
        List of story objects from that date
    """
    endpoints = {
        'top': 'https://hacker-news.firebaseio.com/v0/topstories.json',
        'new': 'https://hacker-news.firebaseio.com/v0/newstories.json',
        'best': 'https://hacker-news.firebaseio.com/v0/beststories.json',
        'ask': 'https://hacker-news.firebaseio.com/v0/askstories.json',
        'show': 'https://hacker-news.firebaseio.com/v0/showstories.json',
        'job': 'https://hacker-news.firebaseio.com/v0/jobstories.json'
    }
    
    response = requests.get(endpoints[story_type])
    story_ids = response.json()
    
    # Calculate start and end of target date
    start_of_day = target_date.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day + timedelta(days=1)
    
    start_timestamp = int(start_of_day.timestamp())
    end_timestamp = int(end_of_day.timestamp())
    
    filtered_stories = []
    
    print(f"Fetching stories for {target_date.strftime('%Y-%m-%d')}...")
    
    for i, story_id in enumerate(story_ids):
        try:
            story_response = requests.get(
                f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            )
            story = story_response.json()
            
            if story and 'time' in story:
                story_time = story['time']
                
                # Check if story is within the specific date
                if start_timestamp <= story_time < end_timestamp:
                    filtered_stories.append(story)
                    print(f"✓ Found: {story.get('title', 'No title')[:60]}...")
            
            time.sleep(0.1)
            
            if (i + 1) % 50 == 0:
                print(f"Processed {i + 1} stories, found {len(filtered_stories)} matches")
                
        except Exception as e:
            print(f"Error fetching story {story_id}: {e}")
            continue
    
    return filtered_stories


# Example usage:

# Get top stories from the last 24 hours
today_stories = get_stories_by_timeframe(story_type='top', days_back=1)

# Get top stories from the last week
week_stories = get_stories_by_timeframe(story_type='top', days_back=7)

# Get stories from a specific date
specific_date = datetime(2026, 1, 30)  # January 30, 2026
specific_day_stories = get_stories_for_specific_date(specific_date, story_type='top')

# Print results
print(f"\nToday's stories: {len(today_stories)}")
print(f"This week's stories: {len(week_stories)}")
print(f"Stories from {specific_date.strftime('%Y-%m-%d')}: {len(specific_day_stories)}")

Fetching stories from the last 1 day(s)...
Cutoff timestamp: 1769751439 (2026-01-30 11:07:19.645053)
✓ Found: Antirender: remove the glossy shine on architectural renderi...
✓ Found: Show HN: I trained a 9M speech model to fix my Mandarin tone...
✓ Found: Peerweb: Decentralized website hosting via WebTorrent...
✓ Found: Stonebraker on CAP theorem and Databases (2010)...
✓ Found: Kimi K2.5 Technical Report [pdf]...

Reached stories older than 1 day(s). Stopping...
Fetching stories from the last 7 day(s)...
Cutoff timestamp: 1769233045 (2026-01-24 11:07:25.436813)
✓ Found: Antirender: remove the glossy shine on architectural renderi...
✓ Found: Show HN: I trained a 9M speech model to fix my Mandarin tone...
✓ Found: Peerweb: Decentralized website hosting via WebTorrent...
✓ Found: Stonebraker on CAP theorem and Databases (2010)...
✓ Found: Kimi K2.5 Technical Report [pdf]...
✓ Found: Disrupting the largest residential proxy network...
✓ Found: The $100B megadeal between OpenAI and Nvidia